### Fully Connected Neural Network - Classification

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
import pandas as pd
import requests
import collections
import numpy as np

full_data = pd.read_csv('f1_data.csv')

2023-07-31 20:15:57.484900: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def create_classification_model(num_features, units=(128, 64), activation='relu', dropout_rate=0.0):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(units[0], activation=activation, input_shape=(num_features,)),
        tf.keras.layers.Dense(units[1], activation=activation),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(1, activation='sigmoid')  # Adjusted to 1 unit for binary classification
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy')
    return model


In [3]:
# Using formuladata api

def get_location_details(location):
    url = 'https://formuladataapi.pythonanywhere.com/api/f1/circuit_data'
    filters = {}
    filters['location'] = location
    response = requests.get(url, params=filters)
    data = response.json()
    try:
      latitude = data[0]['latitude']
      longitude = data[0]['longitude']
      circuit_length = float(data[0]['circuit_length'][0:3])
    except:
      return None
    return [latitude, longitude, circuit_length]


def get_fp_details(driver, season, round):
    url = 'https://formuladataapi.pythonanywhere.com/api/f1'
    filters = {}
    filters['driver_name'] = driver
    filters['round'] = round
    filters['season'] = season
    response = requests.get(url, params=filters)
    data = response.json()
    try:
      fp1 = int(data[0]['fp1_position'])
    except:
      fp1 = None
    try:
      fp2 = int(data[0]['fp2_position'])
    except:
      fp2 = None
    try:
      fp3 = int(data[0]['fp3_position'])
    except:
      fp3 = None
    return [fp1, fp2, fp3]

In [4]:
def get_race_results_with_fp(map, season, round, location, weather, XX, model, fps):
    race_results = {}
    weather_dict = {'dry':0, "cloudy":1, "wet":2}
    latitude = get_location_details(location)[0]
    longitude = get_location_details(location)[1]
    circuit_length = get_location_details(location)[2]

    for driver, team in map.items():
        datapoint = [0]*XX.shape[1]
        datapoint[0] = season
        datapoint[1] = round
        datapoint[2] = weather_dict[weather]
        try:
            datapoint[3] = int(fps[driver][0])
        except:
            datapoint[3] = 20
        try:
            datapoint[4] = int(fps[driver][1])
        except:
            datapoint[4] = 20
        try:
            datapoint[5] = int(fps[driver][2])
        except:
            datapoint[5] = 20
        datapoint[6] = circuit_length
        datapoint[7] = latitude
        datapoint[8] = longitude

        loc = location.lower().replace(' ', '_')
        location_index = XX.columns.get_loc(f'location_{loc}')
        datapoint[location_index] = 1
    
        driver_index = XX.columns.get_loc(f'driver_name_{driver}')
        team_index = XX.columns.get_loc(f'constructor_name_{team}')
        datapoint[driver_index] = 1
        datapoint[team_index] = 1

        df = pd.DataFrame([datapoint], columns=XX.columns)
        test_prediction = model.predict(df ,verbose=0)
        race_results[driver] = test_prediction[0][0]
    sorted_results = sorted(race_results.items(), key=lambda x: x[1], reverse = True)
    sorted_results = collections.OrderedDict(sorted_results)
    return sorted_results

In [5]:
driver_team_mapping = {
    'max_verstappen': 'red_bull_racing',
    'fernando_alonso': 'aston_martin',
    'lewis_hamilton': 'mercedes',
    'charles_leclerc': 'ferrari',
    'carlos_sainz': 'ferrari',
    'sergio_perez': 'red_bull_racing',
    'alexander_albon': 'williams',
    'esteban_ocon': 'aston_martin',
    'lance_stroll': 'aston_martin',
    'valtteri_bottas': 'alfa_romeo',
    'oscar_piastri': 'mclaren',
    'pierre_gasly': 'renault',
    'lando_norris': 'mclaren',
    'yuki_tsunoda': 'toro_rosso',
    'nico_hulkenberg': 'haas',
    'zhou_guanyu': 'alfa_romeo',
    'kevin_magnussen': 'haas',
    'nyck_de_vries': 'toro_rosso',
    'george_russell': 'mercedes',
    'logan_sargeant': 'williams'
}

In [7]:
fps = {}
for driver, _ in driver_team_mapping.items():
    drivers = driver.split('_')
    for d in range(len(drivers)):
        drivers[d] = drivers[d][0].upper() + drivers[d][1:]
    driver_parsed = ' '.join(drivers)
    fps[driver] = get_fp_details(driver_parsed, 2023, 10)

print(fps)

{'max_verstappen': [1, 1, 8], 'fernando_alonso': [4, 10, 3], 'lewis_hamilton': [12, 15, 5], 'charles_leclerc': [5, None, 1], 'carlos_sainz': [7, 2, 6], 'sergio_perez': [2, 4, 14], 'alexander_albon': [3, 3, 2], 'esteban_ocon': [6, 13, 16], 'lance_stroll': [9, 6, 13], 'valtteri_bottas': [15, 16, 18], 'oscar_piastri': [10, 9, 17], 'pierre_gasly': [13, 8, 4], 'lando_norris': [8, 14, 12], 'yuki_tsunoda': [16, 18, 10], 'nico_hulkenberg': [20, 7, 19], 'zhou_guanyu': [18, 11, 20], 'kevin_magnussen': [19, 17, 15], 'nyck_de_vries': [11, 19, 11], 'george_russell': [14, 12, 9], 'logan_sargeant': [17, 5, 7]}


##### Race Prediction

In [8]:
race_data = full_data.iloc[:]
race_data["in_top_5"] = race_data['race_finishing_position'].apply(lambda x: 1 if x<=5 else 0)
race_data = race_data.drop(["grid_position", "has_fastest_lap","race_laps_completed","points", "fastest_lap_position", "race_finishing_position"], axis = 1)

# 2021: 14744

X = race_data.drop('in_top_5', axis=1)  
y = race_data['in_top_5']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=25)
num_features = X_train.shape[1]
print("Training data shape:", X_train.shape, y_train.shape)
print("Testing data shape:", X_test.shape, y_test.shape)

Training data shape: (12659, 416) (12659,)
Testing data shape: (3165, 416) (3165,)


In [9]:
race_model = create_classification_model(num_features, (64, 32), 'relu', 0.1)
race_model.fit(X_train, y_train, epochs=100, batch_size=32)

accuracy = race_model.evaluate(X_test, y_test)
predictions = race_model.predict(X_test)
predicted_labels = np.argmax(predictions, axis=1)
accuracy = accuracy_score(y_test, predicted_labels)
print('Accuracy:', accuracy)

Epoch 1/100
396/396 [==============================] - 2s 1ms/step - loss: 2.8945
Epoch 2/100
396/396 [==============================] - 1s 2ms/step - loss: 0.4655
Epoch 3/100
396/396 [==============================] - 1s 1ms/step - loss: 0.4519
Epoch 4/100
396/396 [==============================] - 1s 2ms/step - loss: 0.4381
Epoch 5/100
396/396 [==============================] - 1s 2ms/step - loss: 0.4234
Epoch 6/100
396/396 [==============================] - 1s 2ms/step - loss: 0.4221
Epoch 7/100
396/396 [==============================] - 1s 1ms/step - loss: 0.4161
Epoch 8/100
396/396 [==============================] - 1s 1ms/step - loss: 0.4033
Epoch 9/100
396/396 [==============================] - 1s 2ms/step - loss: 0.4031
Epoch 10/100
396/396 [==============================] - 1s 2ms/step - loss: 0.3999
Epoch 11/100
396/396 [==============================] - 1s 2ms/step - loss: 0.3977
Epoch 12/100
396/396 [==============================] - 1s 2ms/step - loss: 0.3980
Epoch 13/100


In [10]:
results_race = get_race_results_with_fp(driver_team_mapping, 2023, 10, 'Great Britain', 'cloudy', X_test, race_model, fps)
arr_race = []

for key, val in results_race.items():
    arr_race.append((key,"{:.2f}".format(float(val))))


for item in arr_race:
    print(item[0], item[1])

max_verstappen 0.79
alexander_albon 0.66
sergio_perez 0.64
carlos_sainz 0.59
lewis_hamilton 0.56
fernando_alonso 0.54
charles_leclerc 0.46
george_russell 0.40
pierre_gasly 0.31
logan_sargeant 0.29
oscar_piastri 0.27
lando_norris 0.26
lance_stroll 0.13
esteban_ocon 0.02
valtteri_bottas 0.01
nico_hulkenberg 0.01
kevin_magnussen 0.00
yuki_tsunoda 0.00
nyck_de_vries 0.00
zhou_guanyu 0.00


##### Qualifying Prediction

In [11]:
quali_data = full_data.iloc[:]
quali_data["in_top_5"] = quali_data['grid_position'].apply(lambda x: 1 if x<=5 else 0)
quali_data = quali_data.drop(["grid_position", "has_fastest_lap","race_laps_completed","points", "fastest_lap_position", "race_finishing_position"], axis = 1)

# 2021: 14744

X = quali_data.drop('in_top_5', axis=1)  
y = quali_data['in_top_5']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=25)
num_features = X_train.shape[1]
print("Training data shape:", X_train.shape, y_train.shape)
print("Testing data shape:", X_test.shape, y_test.shape)

Training data shape: (12659, 416) (12659,)
Testing data shape: (3165, 416) (3165,)


In [12]:
quali_model = create_classification_model(num_features, (64, 32), 'relu', 0.1)
quali_model.fit(X_train, y_train, epochs=100, batch_size=32)

accuracy = quali_model.evaluate(X_test, y_test)
predictions = quali_model.predict(X_test)
predicted_labels = np.argmax(predictions, axis=1)
accuracy = accuracy_score(y_test, predicted_labels)
print('Accuracy:', accuracy)

Epoch 1/100
396/396 [==============================] - 2s 2ms/step - loss: 2.9287
Epoch 2/100
396/396 [==============================] - 1s 2ms/step - loss: 0.3875
Epoch 3/100
396/396 [==============================] - 1s 2ms/step - loss: 0.3716
Epoch 4/100
396/396 [==============================] - 1s 2ms/step - loss: 0.3651
Epoch 5/100
396/396 [==============================] - 1s 3ms/step - loss: 0.3436
Epoch 6/100
396/396 [==============================] - 1s 4ms/step - loss: 0.3457
Epoch 7/100
396/396 [==============================] - 1s 2ms/step - loss: 0.3553
Epoch 8/100
396/396 [==============================] - 1s 2ms/step - loss: 0.3561
Epoch 9/100
396/396 [==============================] - 1s 2ms/step - loss: 0.3489
Epoch 10/100
396/396 [==============================] - 1s 2ms/step - loss: 0.3470
Epoch 11/100
396/396 [==============================] - 1s 2ms/step - loss: 0.3383
Epoch 12/100
396/396 [==============================] - 1s 2ms/step - loss: 0.3321
Epoch 13/100


In [13]:
results_quali = get_race_results_with_fp(driver_team_mapping, 2023, 10, 'Great Britain', 'cloudy', X_test, quali_model, fps)
arr_quali = []

for key, val in results_quali.items():
    arr_quali.append((key,"{:.2f}".format(float(val))))


for item in arr_quali:
    print(item[0], item[1])

max_verstappen 0.90
alexander_albon 0.86
sergio_perez 0.80
carlos_sainz 0.68
lewis_hamilton 0.46
fernando_alonso 0.42
logan_sargeant 0.41
charles_leclerc 0.36
pierre_gasly 0.31
george_russell 0.19
lando_norris 0.14
oscar_piastri 0.14
lance_stroll 0.10
nico_hulkenberg 0.02
esteban_ocon 0.02
valtteri_bottas 0.01
kevin_magnussen 0.01
nyck_de_vries 0.00
zhou_guanyu 0.00
yuki_tsunoda 0.00


##### Fastest Lap Prediction

In [14]:
fl_data = full_data.iloc[:]
fl_data["in_top_5"] = fl_data['fastest_lap_position'].apply(lambda x: 1 if x<=5 else 0)
fl_data = fl_data.drop(["grid_position", "has_fastest_lap","race_laps_completed","points", "fastest_lap_position", "race_finishing_position"], axis = 1)

# 2021: 14744

X = fl_data.drop('in_top_5', axis=1)  
y = fl_data['in_top_5']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=25)
num_features = X_train.shape[1]
print("Training data shape:", X_train.shape, y_train.shape)
print("Testing data shape:", X_test.shape, y_test.shape)

Training data shape: (12659, 416) (12659,)
Testing data shape: (3165, 416) (3165,)


In [15]:
fl_model = create_classification_model(num_features, (64, 32), 'relu', 0.1)
fl_model.fit(X_train, y_train, epochs=100, batch_size=32)

accuracy = fl_model.evaluate(X_test, y_test)
predictions = fl_model.predict(X_test)
predicted_labels = np.argmax(predictions, axis=1)
accuracy = accuracy_score(y_test, predicted_labels)
print('Accuracy:', accuracy)

Epoch 1/100
396/396 [==============================] - 2s 2ms/step - loss: 1.1790
Epoch 2/100
396/396 [==============================] - 1s 2ms/step - loss: 0.4426
Epoch 3/100
396/396 [==============================] - 1s 2ms/step - loss: 0.4487
Epoch 4/100
396/396 [==============================] - 1s 2ms/step - loss: 0.4368
Epoch 5/100
396/396 [==============================] - 1s 3ms/step - loss: 0.4292
Epoch 6/100
396/396 [==============================] - 1s 3ms/step - loss: 0.4261
Epoch 7/100
396/396 [==============================] - 1s 2ms/step - loss: 0.4330
Epoch 8/100
396/396 [==============================] - 1s 2ms/step - loss: 0.4255
Epoch 9/100
396/396 [==============================] - 1s 3ms/step - loss: 0.4229
Epoch 10/100
396/396 [==============================] - 1s 2ms/step - loss: 0.4207
Epoch 11/100
396/396 [==============================] - 1s 2ms/step - loss: 0.4228
Epoch 12/100
396/396 [==============================] - 1s 2ms/step - loss: 0.4214
Epoch 13/100


In [16]:
results_fl = get_race_results_with_fp(driver_team_mapping, 2023, 10, 'Great Britain', 'cloudy', X_test, fl_model, fps)
arr_fl = []

for key, val in results_fl.items():
    arr_fl.append((key,"{:.2f}".format(float(val))))


for item in arr_fl:
    print(item[0], item[1])

max_verstappen 0.76
alexander_albon 0.74
sergio_perez 0.73
carlos_sainz 0.66
lewis_hamilton 0.58
logan_sargeant 0.55
fernando_alonso 0.47
george_russell 0.41
charles_leclerc 0.33
pierre_gasly 0.30
lando_norris 0.29
oscar_piastri 0.27
lance_stroll 0.17
yuki_tsunoda 0.06
zhou_guanyu 0.05
nico_hulkenberg 0.03
valtteri_bottas 0.03
kevin_magnussen 0.02
nyck_de_vries 0.02
esteban_ocon 0.01


In [17]:
print("RACE ", arr_race)
print("........................")
print("QUALI", arr_quali)
print("........................")
print("F_LAP", arr_fl)

RACE  [('max_verstappen', '0.79'), ('alexander_albon', '0.66'), ('sergio_perez', '0.64'), ('carlos_sainz', '0.59'), ('lewis_hamilton', '0.56'), ('fernando_alonso', '0.54'), ('charles_leclerc', '0.46'), ('george_russell', '0.40'), ('pierre_gasly', '0.31'), ('logan_sargeant', '0.29'), ('oscar_piastri', '0.27'), ('lando_norris', '0.26'), ('lance_stroll', '0.13'), ('esteban_ocon', '0.02'), ('valtteri_bottas', '0.01'), ('nico_hulkenberg', '0.01'), ('kevin_magnussen', '0.00'), ('yuki_tsunoda', '0.00'), ('nyck_de_vries', '0.00'), ('zhou_guanyu', '0.00')]
........................
QUALI [('max_verstappen', '0.90'), ('alexander_albon', '0.86'), ('sergio_perez', '0.80'), ('carlos_sainz', '0.68'), ('lewis_hamilton', '0.46'), ('fernando_alonso', '0.42'), ('logan_sargeant', '0.41'), ('charles_leclerc', '0.36'), ('pierre_gasly', '0.31'), ('george_russell', '0.19'), ('lando_norris', '0.14'), ('oscar_piastri', '0.14'), ('lance_stroll', '0.10'), ('nico_hulkenberg', '0.02'), ('esteban_ocon', '0.02'), ('v

In [20]:
# !pip install tensorflowjs

  Using cached tensorflowjs-4.9.0-py3-none-any.whl (89 kB)
  Using cached tensorflow_decision_forests-1.5.0-cp310-cp310-macosx_10_15_x86_64.whl (12.3 MB)
  Using cached importlib_resources-6.0.0-py3-none-any.whl (31 kB)
  Using cached jax-0.4.13-py3-none-any.whl
  Using cached tensorflow_hub-0.14.0-py2.py3-none-any.whl (90 kB)
  Using cached packaging-20.9-py2.py3-none-any.whl (40 kB)
  Using cached flax-0.6.2-py3-none-any.whl (189 kB)
  Using cached rich-13.4.2-py3-none-any.whl (239 kB)
  Using cached tensorstore-0.1.40-cp310-cp310-macosx_10_14_x86_64.whl (13.8 MB)
  Using cached optax-0.1.5-py3-none-any.whl (164 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.4/189.4 KB 3.1 MB/s eta 0:00:00a 0:00:01
  Using cached msgpack-1.0.5-cp310-cp310-macosx_10_9_x86_64.whl (74 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 7.9 MB/s eta 0:00:0000:0100:01
  Using cached ml_dtypes-0.2.0-cp310-cp310-macosx_10_9_universal2.whl (1.2 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [21]:
# import tensorflowjs as tfjs

In [24]:
# tfjs.converters.save_keras_model(race_model, "/Users/anirudhkrishna/GitHub/FormulaData/data-modelling/race_model/")
# tfjs.converters.save_keras_model(quali_model, "/Users/anirudhkrishna/GitHub/FormulaData/data-modelling/quali_model/")
# tfjs.converters.save_keras_model(fl_model, "/Users/anirudhkrishna/GitHub/FormulaData/data-modelling/fl_model/")


In [26]:
# /Users/anirudhkrishna/GitHub/FormulaData/api/

# race_model.save("/Users/anirudhkrishna/GitHub/FormulaData/api/race_model.h5")
# quali_model.save("/Users/anirudhkrishna/GitHub/FormulaData/api/quali_model.h5")
# fl_model.save("/Users/anirudhkrishna/GitHub/FormulaData/api/fl_model.h5")

/Users/anirudhkrishna/GitHub/FormulaData/formulavenv/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# full_data.iloc[-20:].to_csv("/Users/anirudhkrishna/GitHub/FormulaData/api/sample_data.csv", index=False)